In [ ]:
import pickle

with open('/home/gk/Downloads/Telegram Desktop/pair_dists_reversed_2.pkl', 'rb') as f:
    my_dict = pickle.load(f)

In [ ]:
my_dict

In [ ]:
import networkx as nx

G = nx.read_graphml('/home/gk/code/migration_project/G12.graphml')

In [ ]:
G.nodes(data=True)

In [ ]:
import geopandas as gpd
from shapely.wkt import loads

# assuming you already have a graph G with node data that includes a "geometry" attribute
# use the nodes method from networkx to get a NodeDataView object
nodedata = G.nodes(data=True)

# create a list of dictionaries to store the attributes of each node
node_attrs = []
for n, data in nodedata:
    node_dict = data.copy()
    node_dict["nodeID"] = n
    node_dict["geometry"] = loads(node_dict["geometry"])
    node_attrs.append(node_dict)

# create a GeoDataFrame from the list of node attributes, using the "geometry" attribute as the geometry
nodes_gdf = gpd.GeoDataFrame(node_attrs, geometry='geometry', crs=4326)

In [ ]:
nodes_gdf

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from tqdm import tqdm

dist_dict = my_dict
gdf = nodes_gdf

# create a list to store the lines
lines = []

# loop through the keys and values in the dictionary
for k, v in tqdm(dist_dict.items()):
    # check if the distance is less than 100 km
    if v < 100:
        # get the two points from the gdf based on the indexes in the key tuple
        point1 = gdf.iloc[k[0]]["geometry"]
        point2 = gdf.iloc[k[1]]["geometry"]
        # create a LineString between the two points and append it to the lines list
        lines.append(LineString([point1, point2]))

# create a new gdf with the lines
new_gdf = gpd.GeoDataFrame(
    pd.DataFrame(
        {"distance": [dist_dict[k] for k in dist_dict.keys() if dist_dict[k] < 100]}
    ),
    geometry=lines,
)

# save the new gdf as a geojson file
new_gdf.to_file("new_gdf_2.geojson", driver="GeoJSON")